## Manual CPE matching evaluation

This notebook assists the manual matching evaluation

In [1]:
from sec_certs.dataset import CCDataset
import pandas as pd
import json
import tempfile

## Prepare the input data for label studio

In [2]:
dset = CCDataset.from_web_latest()
df = dset.to_pandas()

eval_digests = pd.read_csv("./../../data/cpe_eval/random.csv", sep=";").set_index("dgst").index
eval_certs = df.loc[(df.index.isin(eval_digests)) & (df.cpe_matches.notnull())].copy()

# It may be handy to display max number of assigned cpe matches here
eval_certs["n_cpes"] = eval_certs.cpe_matches.map(len)
max_n_cpes = eval_certs.n_cpes.max()
print(f"Max CPE matches: {max_n_cpes}")

# Now you may want to adjust the key `cpe_n_max_matches` config in sec_certs/config/settings.yml according to max_n_cpes
# This helps to avoid clutter in label studio interface
with tempfile.TemporaryDirectory() as tmp_dir:
    dset.root_dir = tmp_dir
    dset.certs = {x.dgst: x for x in dset if x.dgst in eval_certs.index.tolist()}
    dset.to_label_studio_json("./label_studio_input_data.json", update_json=False)

`Now you import this data to label studio and label it`

## Load the data from label studio and show the results

In [4]:
with open("./../../data/cpe_eval/manual_cpe_labels.json", "r") as handle:
    data = json.load(handle)

results = []
for sample in data:
    option_keys = [key for key in sample.keys() if "option_" in key]
    n_cpe_matches = len([sample[key] for key in option_keys if sample[key] != "No good match"])

    if not "verified_cpe_match" in sample.keys():
        n_wrong = 0
    elif isinstance(sample["verified_cpe_match"], str):
        n_wrong = 1
    else:
        n_wrong = len(sample["verified_cpe_match"]["choices"])

    results.append((n_cpe_matches, n_wrong))

correct = [x[0] - x[1] for x in results]
wrong = [x[1] for x in results]
n_candidates = [x[0] for x in results]
completely_right = [x == 0 for x in wrong]

precision = 100 * sum(correct) / sum(n_candidates)
completely_right_ratio = 100 * sum(completely_right) / len(n_candidates)

print(f"Evaluated {sum(n_candidates)} CPE matches in {len(results)} certificates")
print(f"In total, {sum(correct)} ({precision:.2f}%) are correct (precision of the positive class).")
print(f"Also, {sum(completely_right)} ({completely_right_ratio:.2f}%) certificates have perfect matches.")

print(f"\\newcommand{{\\evalCcPrecision}}{{${precision:.2f}\%$}}")
print(f"\\newcommand{{\\evalCcRatioErrorFree}}{{${completely_right_ratio:.0f}\%$}}")

Evaluated 607 CPE matches in 100 certificates
In total, 546 (89.95%) are correct (precision of the positive class).
Also, 81 (81.00%) certificates have perfect matches.
\newcommand{\evalCcPrecision}{$89.95\%$}
\newcommand{\evalCcRatioErrorFree}{$81\%$}
